In [1]:
## hyperparameters
struct Hyper
    I::Int # nb of worker types
    J::Int # nb of firm types
end

## structural parameters
Base.@kwdef struct Para
    nu::Vector{Float64} ## distribution of workers per type
    r::Float64
    delta::Vector{Float64}
    eta::Float64
    beta::Float64
    y::Array{Float64,2}
    h::Float64 #h::Vector{Float64}
    k::Vector{Float64}
    #m1::Vector{Float64}
    #s1::Float64
    es::Float64
    xi::Float64
    #rra::Float64
end

Para_etabeta(eta,beta, M::Para) = 
Para(nu=M.nu,r=M.r,delta=M.delta,eta=eta,beta=beta,y=M.y,h=M.h, k=M.k, es=M.es, xi=M.xi)

Para_es(es, M::Para) = 
Para(nu=M.nu,r=M.r,delta=M.delta,eta=M.eta,beta=M.beta,y=M.y,h=M.h, k=M.k, es=es, xi=M.xi)

## define nuc from nu to minimise the number of parameters
nu2nuc(nu) = nu[2:end]./nu[1]
function nuc2nu(nuc)
    den = 1+sum(nuc)
    return(vcat(1,nuc)./den)
end


vectorise(M::Para) = vcat(M.nu,M.r,M.delta,M.eta,M.beta,
                reshape(M.y,length(M.y)), M.h, M.k, M.es, M.xi) 

function Para(vec,N::Hyper)
    nu = vec[1:N.I]
    r = vec[N.I+1]
    index = N.I+1
    
    delta = vec[index .+ (1:N.J)]
    index += N.J
    
    eta = vec[index+1]
    beta = vec[index+2]
    index += 2
    
    y = reshape(vec[index .+ (1:(N.I*N.J))],N.I,N.J)
    index += N.I*N.J
    
    h = vec[index .+ 1]
    index += 1
    
    k = vec[index .+ (1:N.J)]
    index += N.J  
    
    es = vec[index+1]
    xi = vec[index+2]
    
    return(Para(nu,r,delta,eta,beta,y,h, k, es, xi))
end

The search and matching functions. Consider the maximisation $$\max_e s(e)m\Omega-e.$$
I use the closed-form solution when $s(e)=e^{\epsilon}$:
\begin{align}
    & e = (\epsilon m\Omega)^{1/(1-\epsilon)}\\
    & s(e)m\Omega-e = \frac{1-\epsilon}{\epsilon}(\epsilon m\Omega)^{1/(1-\epsilon)}
\end{align}

In [2]:
m(theta,M::Para) = theta^M.eta # M.m1[j] * theta^M.eta
q(theta,M::Para) = theta^(M.eta-1)
#mp(theta,M::Para) =  M.eta *theta^(M.eta-1)) #m(j,theta,M) * M.eta / theta
#qr(theta,M::Para) = theta^(1-M.eta)  #1/M.m1[j] * theta^(1-M.eta) # 1/q
#qrp(theta,M::Para) = #1/M.m1[j] * (1-M.eta) * theta^(-M.eta) # qp/q^2
#qp(j,theta,M::Para) = M.m1[j] * (M.eta-1) * theta^(M.eta-2)


## possible parametrisation s(e) = e^es / (es^es*(1-es)^(1-es))
s(effort,M::Para) = effort^M.es #M.s1 * effort^M.es
#sp(effort,M::Para) = M.s1 * M.es * effort^(M.es-1)
#sp1(z,M::Para)  = (z/(M.s1*M.es))^(1/(M.es-1))

e2s(mX,M::Para) = mX>0 ? (M.es*mX)^(1/(1-M.es)) : 0.
r2s(mX,M::Para) = mX>0 ? (1-M.es)/M.es * (M.es*mX)^(1/(1-M.es)) : 0.
function search(mX,M::Para)
    if mX>0
        e2s = (M.es*mX)^(1/(1-M.es))
        r2s = (1-M.es)/M.es * e2s
        return(e2s,r2s) 
    else
        return(0.,0.)
    end
end

sp1 (generic function with 1 method)

# 1) Define allocations
An Allocation object contains the main endogenous variables:
1. the matrix of match surplus of a worker coming from unemployment,  $\Omega_{ij}^0$,
2. the matrix of unemployed workers' search efforts $e^0_{ij}$
3. the matrix of employed workers' search efforts $e^1_{ij}$
4. the vector of market tightnesses $\theta_j$,
5. the vector for the distribution of unemployed worker $u_i$,
6. the matrix for the distribution of employed workers $n_{ij}$.

In [3]:
struct Alloc{T}
    X::Array{T,2} ##X[i,j] is the surplus W+J+T-U = V-U
    e0::Array{T,2} ## e0[i,j]
    e1::Array{T,3} ## e1[i,j0,j1] is effort from j0 to j1
    theta::Vector{T} ## theta[j]
    u::Vector{T} ## u[i]
    n::Array{T,2} ## n[i,j]
end

function Alloc(x::Float64,N::Hyper)
    X = fill(x,N.I,N.J) 
    e0 = fill(x,N.I,N.J)
    e1 = fill(x,N.I,N.J,N.J)
    theta = fill(x,N.J)
    n = fill(x, N.I, N.J)
    u = fill(x, N.I)
    Alloc(X,e0,e1,theta,u,n)
end

vectorise(A::Alloc)= vcat(reshape(A.X,length(A.X)),reshape(A.e0,length(A.e0)),
    reshape(A.e1,length(A.e1)),reshape(A.theta,length(A.theta)),
    reshape(A.u,length(A.u)),reshape(A.n,length(A.n)))

function Alloc(vec::Array{Float64},N::Hyper)
    A = Alloc(0.,N)
    Xvec = vec[1:(N.I*N.J)]
    index = length(Xvec)
    e0vec= vec[index .+ (1:(N.I*N.J))]
    index += length(e0vec)
    e1vec= vec[index .+ (1:(N.I*N.J*N.J))]
    index += length(e1vec)
    thetavec= vec[index .+ (1:N.J)]
    index += length(thetavec)
    uvec= vec[index .+ (1:N.I)]
    index += length(uvec)    
    nvec= vec[index .+ (1:(N.I*N.J))]
    
    A.X[:] = reshape(Xvec, size(A.X))
    A.e0[:] = reshape(e0vec, size(A.e0))
    A.e1[:] = reshape(e1vec, size(A.e1))
    A.theta[:] = reshape(thetavec, size(A.theta))
    A.u[:] = reshape(uvec, size(A.u))
    A.n[:] = reshape(nvec, size(A.n))
    
    return(A)
end

allocize (generic function with 1 method)

Before writing the objective function to compute the decentralized equilibrium, I define the Taxes object.
Each job is taxed $T_{ij}=\frac{t_{ij}}{r+\delta_j}$. 
\begin{align}
    &T_{ij}=t^w \frac{w_{ij}}{r+\delta_j}+T^{w,spe}_{ij}+t^y \frac{y_{ij}}{r+\delta_j} + T^X X_{ij}+T^{f0}_j\\
\end{align}
The wage tax is either compensated, $T^{w,spe}_{ij}=-t^w \frac{w_{ij}}{r+\delta_j}$ or uncompensated, $T^{w,spe}_{ij}=0$. 

In the compensated case, $T^{comp}_{ij}=t^y \frac{y_{ij}}{r+\delta_j} + T^X X_{ij}+T^{f0}_j$.

An uncompensated wage tax is equivalent to a combination of a tax on $y$ and on $X$ (plus the effect on $\tilde{\beta}$). We use the wage equation in structure.ipynb to obtain a compensated equivalent:
\begin{align}
    &T^{uncomp}_{ij}=\left(t^y+\frac{t^w}{1+\tilde{\beta}t^w} (1-\tilde{\beta}t^y)\right) \frac{y_{ij}}{r+\delta_j} + \left(T^X-\frac{t^w}{1+\tilde{\beta}t^w}(1-\tilde{\beta}+\tilde{\beta} T^X)\right) X_{ij}+\left(1-\frac{t^w}{1+\tilde{\beta}t^w}\tilde{\beta}\right)T^{f0}_j\\
\end{align}

For each job transition, there is a tax $\Theta^{j_0}_{ij_1}$.
\begin{align}
    & \Theta^{j_0}_{ij_1}=t^w B^{j_0}_{ij_1}+\Theta^{spe,j_0}_{ij_1}+\Theta^{f1}_{j_0}=\Theta^{f1}_{j_0}
\end{align}
I assume that this tax is always compensated here, $\Theta^{spe,j_0}_{ij_1}=-t^w B^{j_0}_{ij_1}$. This critically simplifies the algorithm to compute the decentralised allocation.
We define $$TT^{j_0}_{ij_1}=\Theta^{j_0}_{ij_1}+T_{ij_1}-T_{ij_0}.$$

The Taxes object contains information on $(t^w,t^y,T^X,T^{f0}_{j},\Theta^{f1}_{j})$, assuming that it corresponds to a compensated tax system.


With these notations, the optimal tax scheme is always compensated with 
\begin{align}
    & T_{ij}^{0}=\frac{t_{ij}^0}{r+\delta_j}=T^{f0}_j+T^{w,spe}_{ij},\\
    & \mathcal{T}_{ij_1}^{j_0} = \Theta^{f1}_{j_0}+\Theta^{spe,j_0}_{ij_1}.
\end{align}



In [4]:
struct Taxes{T <: Real}
    w::T ## tauw compensated by default
    y::T ## tauy
    X::T ## T^X
    f0::Vector{T}
    f1::Vector{T}
end

## function to obtain a compensated equivalent
function comp_Taxes(T::Taxes,M::Para)
    betatilde = M.beta / (M.beta + (1-M.beta)*(1+T.w))
    fw = T.w/(1+betatilde*T.w) 
    NTy = T.y + fw * (1-betatilde*T.y)
    NTX = T.X - fw * (1- betatilde + betatilde*T.X)
    NTf0 = T.f0 .* (1-fw*betatilde)
    Taxes(T.w,NTy,NTX,NTf0,T.f1) 
end
    

## taxes raised from a match (i,j) and from a match(i,j0,j1)
## /!\ compensated
function T_tax(i,j, X::Array{Float64}, T::Taxes, M::Para) 
    T.y * (M.y[i,j] / (M.r+M.delta[j])) + T.X * X[i,j] + T.f0[j]
end
function T_tax_vec(j, yi::Vector{Float64},Xi::Vector{Float64}, T::Taxes, M::Para )
    T.y * (yi[j]/ (M.r+M.delta[j])) + T.X * Xi[j] + T.f0[j]
end
function T_tax_sca(j, yij::Float64, Xij::Float64, T::Taxes, M::Para )
    T.y * (yij/ (M.r+M.delta[j])) + T.X * Xij + T.f0[j]
end


TT_tax(i,j0,j1, X::Array{Float64},T::Taxes, M::Para) =  
        T_tax(i,j1,X,T,M) - T_tax(i,j0,X,T,M) + T.f1[j0]
TT_tax_vec(j0,j1,yi::Vector{Float64}, Xi::Vector{Float64},T::Taxes, M::Para) =  
        T_tax_vec(j1,yi,Xi,T,M) - T_tax_vec(j0,yi,Xi,T,M) + T.f1[j0] 

TT_tax (generic function with 1 method)

Define a Bound object for the calibration.

In [ ]:
Base.@kwdef struct Bounds
    nuc::Float64
    y::Float64
    alpha::Float64
    theta::Float64
    es::Float64
    xi::Float64
end

struct Moments
    mom1::Vector{Float64}
    mom2::Array{Float64,2}
    mom3::Array{Float64,2}
    mom4::Vector{Float64}
end

struct ParCalib
    emp::Moments
    wgt::Vector{Float64}
    lbd::Bounds
    ubd::Bounds
end

## 2) Save and open

In [ ]:
function save_struc(A::Alloc, M::Para, file, VER, N::Hyper)  
    col = DataFrame(x = vcat(vectorise(M),vectorise(A)) ) 
    rename!(col,[Symbol(VER)])
    CSV.write(file, col)
    return()
end

function open_struc(file, N::Hyper)  
    vec = Vector(CSV.read(file)[:,1])
    
    lvec = length(vec)
    lengthA = N.I*N.J + N.I*N.J + N.I*N.J*N.J + N.J + N.I + N.I*N.J
    Mvec = vec[1:(lvec-lengthA)]
    Avec = vec[(lvec-lengthA+1):lvec]
    
    A = Alloc(Avec,N)
    M = Para(Mvec,N)
    return(A,M)
end